In [ ]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import os

# Carregar o modelo pré-treinado ResNet50
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Função para carregar e pré-processar a imagem
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Função para extrair o embedding de uma imagem
def extract_embedding(img_path):
    img = preprocess_image(img_path)
    embedding = model.predict(img)
    return embedding

# Carregar as imagens do banco de dados e extrair embeddings
image_folder = 'path_to_images'  # Pasta com imagens de produtos
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder)]
embeddings = []

for img_path in image_paths:
    embedding = extract_embedding(img_path)
    embeddings.append(embedding)

# Converter a lista de embeddings em uma matriz
embeddings = np.array(embeddings).reshape(len(image_paths), -1)

# Função para recomendar imagens semelhantes
def recommend_image(query_img_path, top_n=5):
    query_embedding = extract_embedding(query_img_path)
    query_embedding = query_embedding.reshape(1, -1)

    similarities = cosine_similarity(query_embedding, embeddings)
    similar_images = similarities.argsort()[0][-top_n:][::-1]  # Ordenar pelas imagens mais semelhantes

    print(f"Imagens semelhantes para: {query_img_path}")
    for idx in similar_images:
        print(image_paths[idx])

# Teste de recomendação
query_img = 'path_to_query_image.jpg'  # Caminho da imagem consultada pelo usuário
recommend_image(query_img)
